In [84]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from numpy.random import multivariate_normal
import seaborn as sns
import matplotlib.pyplot as plt

In [85]:
#loading age-stratified co-occurrence matrices linked to Fyles et al ., 2023
#available at https://github.com/martyn1fyles/CovidSymptomsAnalysisPublic/tree/main/Data

df_u18 = pd.read_csv("co_occurrence_u18.csv", index_col=0)
df_18to54 = pd.read_csv("co_occurrence_18_54.csv", index_col=0)
df_o55 = pd.read_csv("co_occurrence_o55.csv", index_col=0)

In [86]:
#symptom probabilites for each age group, inferred from co-occurrence matrices

symptom_probs_o55 = {
    "Fever": 0.3254,
    "Muscle ache": 0.4305,
    "Fatigue": 0.5915,
    "Sore throat": 0.3088,
    "Cough": 0.6551,
    "Shortness of breath": 0.2548,
    "Headache": 0.4926,
    "Nausea/Vomiting": 0.1938,
    "Abdominal pain": 0.1287,
    "Diarrhoea": 0.1724,
    "Loss of taste": 0.3217,
    "Loss of smell": 0.2669
}

df_symptom_probs_o55 = pd.DataFrame(
    list(symptom_probs_o55.items()), 
    columns=["Symptom", "Occurrence Probability"]
)

print(df_symptom_probs_o55)

                Symptom  Occurrence Probability
0                 Fever                  0.3254
1           Muscle ache                  0.4305
2               Fatigue                  0.5915
3           Sore throat                  0.3088
4                 Cough                  0.6551
5   Shortness of breath                  0.2548
6              Headache                  0.4926
7       Nausea/Vomiting                  0.1938
8        Abdominal pain                  0.1287
9             Diarrhoea                  0.1724
10        Loss of taste                  0.3217
11        Loss of smell                  0.2669


In [87]:
symptom_probs_18_54 = {
    "Fever": 0.3472,
    "Muscle ache": 0.4586,
    "Fatigue": 0.5996,
    "Sore throat": 0.3740,
    "Cough": 0.5830,
    "Shortness of breath": 0.2584,
    "Headache": 0.5832,
    "Nausea/Vomiting": 0.1698,
    "Abdominal pain": 0.1128,
    "Diarrhoea": 0.1207,
    "Loss of taste": 0.3903,
    "Loss of smell": 0.3890
}

df_symptom_probs_18_54 = pd.DataFrame(
    list(symptom_probs_18_54.items()), 
    columns=["Symptom", "Occurrence Probability"]
)

print(df_symptom_probs_18_54)

                Symptom  Occurrence Probability
0                 Fever                  0.3472
1           Muscle ache                  0.4586
2               Fatigue                  0.5996
3           Sore throat                  0.3740
4                 Cough                  0.5830
5   Shortness of breath                  0.2584
6              Headache                  0.5832
7       Nausea/Vomiting                  0.1698
8        Abdominal pain                  0.1128
9             Diarrhoea                  0.1207
10        Loss of taste                  0.3903
11        Loss of smell                  0.3890


In [88]:
symptom_probs_u18 = {
    "Fever": 0.3547,
    "Muscle ache": 0.2051,
    "Fatigue": 0.3790,
    "Sore throat": 0.3353,
    "Cough": 0.4218,
    "Shortness of breath": 0.0962,
    "Headache": 0.4568,
    "Nausea/Vomiting": 0.1186,
    "Abdominal pain": 0.0962,
    "Diarrhoea": 0.0583,
    "Loss of taste": 0.2021,
    "Loss of smell": 0.2167
}

df_symptom_probs_u18 = pd.DataFrame(
    list(symptom_probs_u18.items()), 
    columns=["Symptom", "Occurrence Probability"]
)

print(df_symptom_probs_u18)

                Symptom  Occurrence Probability
0                 Fever                  0.3547
1           Muscle ache                  0.2051
2               Fatigue                  0.3790
3           Sore throat                  0.3353
4                 Cough                  0.4218
5   Shortness of breath                  0.0962
6              Headache                  0.4568
7       Nausea/Vomiting                  0.1186
8        Abdominal pain                  0.0962
9             Diarrhoea                  0.0583
10        Loss of taste                  0.2021
11        Loss of smell                  0.2167


In [89]:
#number of individuals in dataset per age group 

N_u18 = 1029
N_18_54 = 5417
N_o55 = 2720

symptom_names = list(symptom_probs_u18.keys())

#marginal probabilities for each symptom(likelihood of occurrence for age group)
marginal_probs_u18 = np.array(list(symptom_probs_u18.values())) 
marginal_probs_18_54 = np.array(list(symptom_probs_18_54.values()))
marginal_probs_o55 = np.array(list(symptom_probs_o55.values()))

In [90]:
#joint probabilities for each age group- co-occurrence matrices as probabilities
joint_probs_u18 = df_u18.values / N_u18 
joint_probs_18_54 = df_18to54.values / N_18_54
joint_probs_o55 = df_o55.values / N_o55

In [91]:
#empty matrices used as placeholders before correlation values are generated
correlation_matrix_u18 = np.zeros_like(joint_probs_u18)
correlation_matrix_18_54 = np.zeros_like(joint_probs_18_54)
correlation_matrix_o55 = np.zeros_like(joint_probs_o55)

In [92]:
#function to calculate correlation matrix based on the given co-occurrences

def compute_correlation_matrix(symptom_names, marginal_probs, joint_probs):
    n = len(symptom_names)
    correlation_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            p_i = marginal_probs[i]
            p_j = marginal_probs[j]
            p_ij = joint_probs[i, j]
            denom = np.sqrt(p_i * (1 - p_i) * p_j * (1 - p_j))
            if denom == 0:
                corr = 0.0
            else:
                corr = (p_ij - p_i * p_j) / denom
            correlation_matrix[i, j] = corr
            
    return correlation_matrix

#apply function to generate correlation matrix 
correlation_matrix_u18 = compute_correlation_matrix(symptom_names, marginal_probs_u18, joint_probs_u18)
correlation_matrix_18_54 = compute_correlation_matrix(symptom_names, marginal_probs_18_54, joint_probs_18_54)
correlation_matrix_o55 = compute_correlation_matrix(symptom_names, marginal_probs_o55, joint_probs_o55)

In [93]:
df_corr_u18 = pd.DataFrame(correlation_matrix_u18, index=symptom_names, columns=symptom_names)
df_corr_18_54 = pd.DataFrame(correlation_matrix_18_54, index=symptom_names, columns=symptom_names)
df_corr_o55 = pd.DataFrame(correlation_matrix_o55, index=symptom_names, columns=symptom_names)


In [94]:
print(df_corr_u18.round(3))

                     Fever  Muscle ache  Fatigue  Sore throat  Cough  \
Fever                1.000        0.207    0.195        0.050  0.070   
Muscle ache          0.207        1.000    0.392        0.215  0.088   
Fatigue              0.195        0.392    1.000        0.209  0.059   
Sore throat          0.050        0.215    0.209        1.000  0.119   
Cough                0.070        0.088    0.059        0.119  1.000   
Shortness of breath  0.089        0.267    0.227        0.152  0.122   
Headache             0.103        0.298    0.285        0.188  0.031   
Nausea/Vomiting      0.143        0.178    0.203        0.121  0.009   
Abdominal pain       0.109        0.169    0.153        0.089 -0.025   
Diarrhoea            0.015        0.079    0.062        0.069 -0.011   
Loss of taste       -0.009        0.146    0.081        0.073 -0.048   
Loss of smell       -0.040        0.066    0.032        0.071 -0.086   

                     Shortness of breath  Headache  Nausea/Vomi

In [95]:
print(df_corr_18_54.round(3))

                     Fever  Muscle ache  Fatigue  Sore throat  Cough  \
Fever                1.000        0.329    0.243        0.189  0.202   
Muscle ache          0.329        1.000    0.438        0.241  0.171   
Fatigue              0.243        0.438    1.000        0.203  0.161   
Sore throat          0.189        0.241    0.203        1.000  0.154   
Cough                0.202        0.171    0.161        0.154  1.000   
Shortness of breath  0.183        0.279    0.304        0.201  0.210   
Headache             0.253        0.347    0.351        0.233  0.106   
Nausea/Vomiting      0.220        0.262    0.266        0.144  0.127   
Abdominal pain       0.186        0.246    0.197        0.132  0.106   
Diarrhoea            0.157        0.201    0.198        0.122  0.117   
Loss of taste        0.092        0.159    0.172        0.073  0.036   
Loss of smell        0.041        0.098    0.134        0.055  0.002   

                     Shortness of breath  Headache  Nausea/Vomi

In [96]:
print(df_corr_o55.round(3))

                     Fever  Muscle ache  Fatigue  Sore throat  Cough  \
Fever                1.000        0.301    0.251        0.152  0.146   
Muscle ache          0.301        1.000    0.436        0.161  0.131   
Fatigue              0.251        0.436    1.000        0.117  0.093   
Sore throat          0.152        0.161    0.117        1.000  0.130   
Cough                0.146        0.131    0.093        0.130  1.000   
Shortness of breath  0.194        0.243    0.297        0.124  0.144   
Headache             0.268        0.363    0.295        0.179  0.068   
Nausea/Vomiting      0.190        0.278    0.273        0.099  0.076   
Abdominal pain       0.185        0.265    0.205        0.123  0.087   
Diarrhoea            0.180        0.211    0.197        0.085  0.071   
Loss of taste        0.135        0.212    0.206        0.069  0.011   
Loss of smell        0.124        0.172    0.163        0.072 -0.001   

                     Shortness of breath  Headache  Nausea/Vomi

In [97]:
#compute standard deviations from marginal probabilities (needed to convert correlation matrices to covariance matrices)
stds_u18 = np.sqrt(marginal_probs_u18 * (1 - marginal_probs_u18))
stds_18_54 = np.sqrt(marginal_probs_18_54 * (1 - marginal_probs_18_54))
stds_o55 = np.sqrt(marginal_probs_o55 * (1 - marginal_probs_o55))

#convert correlation matrices to covariance matrices using matrix multiplication
cov_matrix_u18 = np.diag(stds_u18) @ correlation_matrix_u18 @ np.diag(stds_u18)
cov_matrix_18_54 = np.diag(stds_18_54) @ correlation_matrix_18_54 @ np.diag(stds_18_54)
cov_matrix_o55 = np.diag(stds_o55) @ correlation_matrix_o55 @ np.diag(stds_o55)

In [98]:
np.random.seed(42)

#sample from multivariate distribution

latent_data_u18 = np.random.multivariate_normal(
    mean=np.zeros(len(symptom_names)),
    cov=cov_matrix_u18,
    size=N_u18
)

#convert marginal probabilities to thresholds
thresholds_u18 = norm.ppf(marginal_probs_u18)

#generate binary data- 0 (absent) or 1 (present) depending on if result is below a certain threshold
binary_data_u18 = (latent_data_u18 < thresholds_u18).astype(int)

simulated_df_u18 = pd.DataFrame(binary_data_u18, columns=symptom_names)

print(simulated_df_u18.head())
#simulated_df_u18.to_csv("simulated_data_u18.csv", index=False)

   Fever  Muscle ache  Fatigue  Sore throat  Cough  Shortness of breath  \
0      1            0        0            0      1                    0   
1      0            0        0            0      0                    0   
2      0            0        0            0      0                    0   
3      1            0        0            0      0                    0   
4      0            0        0            0      1                    0   

   Headache  Nausea/Vomiting  Abdominal pain  Diarrhoea  Loss of taste  \
0         0                0               0          0              0   
1         1                0               0          0              0   
2         0                0               0          0              0   
3         1                0               0          0              0   
4         1                0               0          0              0   

   Loss of smell  
0              0  
1              0  
2              0  
3              0  
4        

In [99]:
#repeat process for different age groups

latent_data_18_54 = np.random.multivariate_normal(
    mean=np.zeros(len(symptom_names)),
    cov=cov_matrix_18_54,
    size=N_18_54
)

thresholds_18_54 = norm.ppf(marginal_probs_18_54)

binary_data_18_54 = (latent_data_18_54 < thresholds_18_54).astype(int)

simulated_df_18_54 = pd.DataFrame(binary_data_18_54, columns=symptom_names)

print(simulated_df_18_54.head())
#simulated_df_18_54.to_csv("simulated_data_18_54.csv", index=False)

   Fever  Muscle ache  Fatigue  Sore throat  Cough  Shortness of breath  \
0      0            0        1            0      1                    0   
1      1            0        1            1      1                    0   
2      0            0        1            0      1                    0   
3      0            0        1            1      1                    0   
4      1            1        1            1      0                    1   

   Headache  Nausea/Vomiting  Abdominal pain  Diarrhoea  Loss of taste  \
0         1                0               0          0              0   
1         1                0               0          0              1   
2         1                0               0          0              0   
3         1                0               0          0              0   
4         1                0               0          0              1   

   Loss of smell  
0              0  
1              1  
2              0  
3              0  
4        

In [100]:
latent_data_o55 = np.random.multivariate_normal(
    mean=np.zeros(len(symptom_names)),
    cov=cov_matrix_o55,
    size=N_o55
)

thresholds_o55 = norm.ppf(marginal_probs_o55)

binary_data_o55 = (latent_data_o55 < thresholds_o55).astype(int)

simulated_df_o55 = pd.DataFrame(binary_data_o55, columns=symptom_names)

print(simulated_df_o55.head())
#simulated_df_o55.to_csv("simulated_data_o55.csv", index=False)

   Fever  Muscle ache  Fatigue  Sore throat  Cough  Shortness of breath  \
0      1            0        1            0      1                    0   
1      0            0        1            0      1                    0   
2      0            1        1            0      1                    0   
3      0            1        1            0      1                    0   
4      1            1        1            0      1                    0   

   Headache  Nausea/Vomiting  Abdominal pain  Diarrhoea  Loss of taste  \
0         1                0               0          0              0   
1         1                0               0          0              0   
2         1                0               0          0              0   
3         0                0               0          0              0   
4         0                0               0          0              0   

   Loss of smell  
0              1  
1              0  
2              0  
3              0  
4        

In [101]:
#adding age group labels to label individuals in final dataset

simulated_df_u18['Age'] = 'Under 18'
simulated_df_18_54['Age'] = '18-54'
simulated_df_o55['Age'] = '55 and over'

df = pd.concat([simulated_df_u18, simulated_df_18_54, simulated_df_o55], ignore_index=True)

df.to_csv("simulated_data.csv", index=False)

In [102]:
symptom_columns = [col for col in df.columns if col != "Age"]

#counting number of symptoms occurring at time point 1 per individual
df["SymptomCount_T1"] = df[symptom_columns].sum(axis=1)
grouped = df.groupby("Age")

#counts and proportions of individuals with 5 or more symptoms
summary = grouped["SymptomCount_T1"].apply(lambda x: (x >= 5).sum()).to_frame(name="Count_5_or_more")
summary["Total"] = grouped.size()
summary["Proportion"] = summary["Count_5_or_more"] / summary["Total"]

print(summary)


             Count_5_or_more  Total  Proportion
Age                                            
18-54                   1595   5417    0.294443
55 and over              455   2720    0.167279
Under 18                  13   1029    0.012634


In [103]:
#functions for generating time point 2

def assign_full_symptom_persistence(df, symptom_columns, seed=None):
    if seed is not None:
        np.random.seed(seed)

    #count symptoms present at time point 1 for each individual
    df["SymptomCount_T1"] = df[symptom_columns].sum(axis=1)

    #probabilities of full symptom persistence- dependent on age category and whether 5 or more symptoms were present 
    full_persistence_probs = {
        'Under 18':    {True: 0.21, False: 0.07},
        '18-54':       {True: 0.21, False: 0.07},
        '55 and over': {True: 0.3, False: 0.1}
    }

    def get_prob(row): #high_burden represents 5 or more symptoms being present at time point 1
        age = row["Age"]
        high_burden = row["SymptomCount_T1"] >= 5
        return full_persistence_probs[age][high_burden]

    probabilities = df.apply(get_prob, axis=1)
    df["FullPersistence"] = np.random.binomial(1, probabilities).astype(bool)

    return df


def generate_T2_symptoms(df, symptom_columns, marginal_probs_by_age, persistence_probs, seed=None):
    if seed is not None:
        np.random.seed(seed)

    df_T2 = df.copy()

    for idx, row in df.iterrows():
        age_group = row["Age"]
        full_persist = row["FullPersistence"]
        symptoms_T1 = row[symptom_columns].values.astype(int)
        symptoms_T2 = np.zeros_like(symptoms_T1)

        if full_persist: #if individual is part of full persistence subset, their symptoms are copied across from time point 1 to time point 2
            symptoms_T2 = symptoms_T1.copy()
        else:
            for i, symptom in enumerate(symptom_columns):
                was_present = symptoms_T1[i]
                if was_present: #determines whether symptoms present at time point 1 remain based on defined symptom persistence probabilities
                    p_persist = persistence_probs[symptom]
                    symptoms_T2[i] = np.random.binomial(1, p_persist)
                else:
                    base_prob = marginal_probs_by_age[age_group][symptom] #absent symptoms may become present at time point 2
                    p_appear = 0.1 * base_prob #with a probability 0.1 times the probability the symptom occurred at time point 1 for that age group
                    symptoms_T2[i] = np.random.binomial(1, p_appear)

        for i, symptom in enumerate(symptom_columns):
            df_T2.at[idx, f"{symptom}_T2"] = symptoms_T2[i]

    return df_T2

#defining symptom-specific persistence probabilities based on subject knowledge from research papers and official COVID-19 information from reliable sources
persistence_probs = {
    'Fever': 0.15,
    'Muscle ache': 0.35,
    'Fatigue': 0.65,
    'Sore throat': 0.30,
    'Cough': 0.50,
    'Shortness of breath': 0.60,
    'Headache': 0.40,
    'Nausea/Vomiting': 0.25,
    'Abdominal pain': 0.25,
    'Diarrhoea': 0.30,
    'Loss of taste': 0.55,
    'Loss of smell': 0.55
}

#define marginal probabilities per age group (from T1 simulation)
marginal_probs_by_age = {
    "Under 18": symptom_probs_u18,
    "18-54": symptom_probs_18_54,
    "55 and over": symptom_probs_o55
}

#defining symptom columns
symptom_columns = [
    col for col in df.columns
    if col not in ["Age", "SymptomCount_T1", "FullPersistence"]
    and not col.endswith("_T2")
]

#assign persistence to individuals
df = assign_full_symptom_persistence(df, symptom_columns, seed=42)

#generate T2 symptoms
df = generate_T2_symptoms(df, symptom_columns, marginal_probs_by_age, persistence_probs, seed=42)

#convert T2 columns to binary integers
T2_columns = [col for col in df.columns if col.endswith("_T2")]
df[T2_columns] = df[T2_columns].astype(int)

#df.to_csv("simulated_symptom_data.csv", index=False)